## **Data Wrangling**


<hr>


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m1_survey_data.csv")
df.head(3)

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
1,9,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,New Zealand,No,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,NaN,23.0,Man,No,Bisexual,White or of European descent,No,Appropriate in length,Neither easy nor difficult
2,13,I am a developer by profession,Yes,Less than once a month but more than once per ...,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,Somewhat more welcome now than last year,Tech articles written by other developers;Cour...,28.0,Man,No,Straight / Heterosexual,White or of European descent,Yes,Appropriate in length,Easy


#### Finding and removing duplicates in the dataset

In [3]:
num_rows_before = df.shape[0]
num_duplicate_rows = df.duplicated().sum()
df.drop_duplicates(inplace=True)
num_rows_after = df.shape[0]

if num_rows_before == num_rows_after:
    print("No duplicates were found or all duplicates were already successfully dropped.")
else:
    print("Duplicates were successfully dropped. Number of rows before:", num_rows_before, ", Number of rows after:", num_rows_after,", Number of duplicates dropped:",num_duplicate_rows)


Duplicates were successfully dropped. Number of rows before: 11552 , Number of rows after: 11398 , Number of duplicates dropped: 154


#### Finding Missing values


In [6]:
missing_data = df.isnull()
missing_data.head(5)

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
column = 'WorkLoc'
missing_data_workloc = df['WorkLoc'].isnull()
print(missing_data_workloc.value_counts())
print("")

WorkLoc
False    11366
True        32
Name: count, dtype: int64



#### Imputing missing values


In [13]:
df['WorkLoc'].value_counts()

WorkLoc
Office                                            6806
Home                                              3589
Other place, such as a coworking space or cafe     971
Name: count, dtype: int64

In [17]:
majority_value = df['WorkLoc'].mode()[0]
print("The majority value is: ",majority_value)

The majority value is:  Office


In [18]:
# Replacing and Verification 
df['WorkLoc'].fillna(majority_value, inplace=True)

missing_values_after_imputation = df['WorkLoc'].isnull().sum()

if missing_values_after_imputation == 0:
    print("Imputation successful. No missing values in the 'WorkLoc' column.")
else:
    print("Imputation unsuccessful. There are still missing values in the 'WorkLoc' column.")



Imputation successful. No missing values in the 'WorkLoc' column.


#### Normalizing data


* The dataset contains two columns related to compensation: "CompFreq" indicates payment frequency (Yearly, Monthly, Weekly), while "CompTotal" denotes the amount paid per Year, Month, or Week based on "CompFreq". 
* To facilitate salary comparison, a new column, 'NormalizedAnnualCompensation', is created to standardize annual compensation regardless of payment frequency. This simplifies salary comparisons across developers.


In [28]:
df['CompFreq'].value_counts()

CompFreq
Yearly     6073
Monthly    4788
Weekly      331
Name: count, dtype: int64

In [27]:
# Function to normalize compensation values
def normalize_compensation(val):
    if val['CompFreq'] == 'Yearly':
        return val['CompTotal']
    elif val['CompFreq'] == 'Monthly':
        return val['CompTotal'] * 12
    elif val['CompFreq'] == 'Weekly':
        return val['CompTotal'] * 52
    else:
        return None  

# New column 'NormalizedAnnualCompensation'
df['NormalizedAnnualCompensation'] = df.apply(normalize_compensation, axis=1)


# Verification 
expected_values = df.apply(normalize_compensation, axis=1)
if df['NormalizedAnnualCompensation'].equals(expected_values):
    print("Normalization successful. Values in 'NormalizedAnnualCompensation' match expected values.")
    print(df[['CompFreq', 'CompTotal', 'NormalizedAnnualCompensation']].head())
else:
    print("Normalization unsuccessful. Values in 'NormalizedAnnualCompensation' do not match expected values.")





Normalization successful. Values in 'NormalizedAnnualCompensation' match expected values.
  CompFreq  CompTotal  NormalizedAnnualCompensation
0   Yearly    61000.0                       61000.0
1   Yearly   138000.0                      138000.0
2   Yearly    90000.0                       90000.0
3  Monthly    29000.0                      348000.0
4   Yearly    90000.0                       90000.0
